In [34]:
import pandas as pd
import numpy as np
import datetime as dt
import os, json

In [35]:
def CustomParser1(df):
    j1 = json.loads(df)
    return j1

In [36]:
df=pd.read_csv('Data/Kickstarter.csv',converters={'category':CustomParser1},header=0)
df[sorted(df['category'][0].keys(),reverse=False)] = df['category'].apply(pd.Series)
df1=df[['category','color','parent_id','urls','id','name','position']]
df['category.parent_id']=df1['position']
df['category.id']=df1['color']
df['category.position']=df1['parent_id']
df['category.name']=df1['id']
df['category.slug']=df1['name']
df['category.slug']=df['category.slug'].apply(lambda x: x.split('/'))
df['category.slug']=df['category.slug'].apply(lambda x: x.pop(0))
df[['category','category.parent_id','category.id','category.name','category.position','category.slug']][:1]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 45 columns):
backers_count               3813 non-null int64
blurb                       3813 non-null object
category                    3813 non-null object
converted_pledged_amount    3813 non-null int64
country                     3813 non-null object
created_at                  3813 non-null int64
creator                     3813 non-null object
currency                    3813 non-null object
currency_symbol             3813 non-null object
currency_trailing_code      3813 non-null bool
current_currency            3813 non-null object
deadline                    3813 non-null int64
disable_communication       3813 non-null bool
friends                     0 non-null float64
fx_rate                     3813 non-null float64
goal                        3813 non-null float64
id                          3813 non-null object
is_backing                  0 non-null float64
is_starrable         

In [37]:
def CustomParser2(df2):
    try:
        j2 = json.loads(df2)
        return j2
    except:
        return 0
    pass
            
df2=pd.read_csv('Data/Kickstarter.csv',converters={'creator':CustomParser2},header=0)
droped=df2.loc[df2['creator']==0,['creator']]
df2=df2.loc[df2['creator']!=0]
drop_list=list(droped.index)
#df['creator'].iloc[drop_list]=df['creator'].iloc[drop_list].apply(lambda x: x.replace(' ',','))
df=df.drop(index=drop_list)
print('droped rows:',len(drop_list),drop_list)
print (len(df))
print (len(df2))


droped rows: 3 [587, 1661, 1968]
3810
3810


In [38]:
df2['creator_name']=df2['creator'].apply(lambda x: x['name'])
df2['creator_id']=df2['creator'].apply(lambda x: x['id'])
df['creator_name']=df2['creator_name']
df['creator_id']=df2['creator_id']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3810 entries, 0 to 3812
Data columns (total 47 columns):
backers_count               3810 non-null int64
blurb                       3810 non-null object
category                    3810 non-null object
converted_pledged_amount    3810 non-null int64
country                     3810 non-null object
created_at                  3810 non-null int64
creator                     3810 non-null object
currency                    3810 non-null object
currency_symbol             3810 non-null object
currency_trailing_code      3810 non-null bool
current_currency            3810 non-null object
deadline                    3810 non-null int64
disable_communication       3810 non-null bool
friends                     0 non-null float64
fx_rate                     3810 non-null float64
goal                        3810 non-null float64
id                          3810 non-null object
is_backing                  0 non-null float64
is_starrable         

In [39]:
def clean(df):
    data = df.copy()
    selected_cols = ['creator_name',
                     'creator_id',
                     'backers_count',
                     'blurb',
                     'is_starred',
                     'category.id',
                     'category.name',
                     'category.parent_id',
                     'category.slug',
                     'country',
                     'created_at',
                     'currency',  
                     'deadline',
                     'goal',
                     'launched_at',
                     'spotlight',
                     'staff_pick',
                     'state',
                     'usd_pledged',
                     'usd_type']
    data = data[selected_cols]
    data['is_starred']=data['is_starred'].replace({1: True , None: False})
    data = data.dropna()
    successful = data['state'] == "successful"
    failed = data['state'] == "failed"
    cancelled = data['state'] == "cancelled"
    suspended = data['state'] == "suspended"
    data = data.loc[failed | successful | cancelled | suspended]
    num_cols = ['usd_pledged',
                'deadline',
                'created_at',
                'launched_at']
    data[num_cols] = data[num_cols].apply(pd.to_numeric, errors='coerce')
    data['created_at'] = pd.to_datetime(data['created_at'],unit='s')
    data['launched_at'] = pd.to_datetime(data['launched_at'],unit='s')
    data['deadline'] = pd.to_datetime(data['deadline'],unit='s')
    return data

data = clean(df)


In [40]:
def engineer_features(data):
    data['state'].replace('suspended','failed',inplace=True)   
    data['state_num'] = data['state'].apply(lambda x: 1 if x=='successful' else 0)
    data['launched_at_hr'] = data['launched_at'].apply(lambda x: x.hour) + 1
    data['launched_at_day'] = data['launched_at'].apply(lambda x: x.dayofweek + 1)
    data['launched_at_mo'] = data['launched_at'].apply(lambda x: x.month)
    data['launched_at_yr'] = data['launched_at'].apply(lambda x: x.year)
    data['deadline_hr'] = data['deadline'].apply(lambda x: x.hour) + 1
    data['deadline_day'] = data['deadline'].apply(lambda x: x.dayofweek + 1) 
    data['deadline_mo'] = data['deadline'].apply(lambda x: x.month)
    data['deadline_yr'] = data['deadline'].apply(lambda x: x.year)
    data['created_at_hr'] = data['created_at'].apply(lambda x: x.hour) + 1
    data['created_at_day'] = data['created_at'].apply(lambda x: x.dayofweek + 1) 
    data['created_at_mo'] = data['created_at'].apply(lambda x: x.month)
    data['created_at_yr'] = data['created_at'].apply(lambda x: x.year)
    data['count'] = 1
    data['success'] = (data['state'] == 'successful')
    data['launched-created'] = (data.launched_at - data.created_at).dt.components.days
    data['deadline-launched'] = (data.deadline - data.launched_at).dt.components.days
    return data
data = engineer_features(data)
data.info()
data.to_csv('Data/data.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3173 entries, 0 to 3811
Data columns (total 37 columns):
creator_name          3173 non-null object
creator_id            3173 non-null int64
backers_count         3173 non-null int64
blurb                 3173 non-null object
is_starred            3173 non-null object
category.id           3173 non-null int64
category.name         3173 non-null object
category.parent_id    3173 non-null float64
category.slug         3173 non-null object
country               3173 non-null object
created_at            3173 non-null datetime64[ns]
currency              3173 non-null object
deadline              3173 non-null datetime64[ns]
goal                  3173 non-null float64
launched_at           3173 non-null datetime64[ns]
spotlight             3173 non-null bool
staff_pick            3173 non-null bool
state                 3173 non-null object
usd_pledged           3173 non-null float64
usd_type              3173 non-null object
state_num    